In [1]:
import os

In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   54G   47G  54% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   54G   47G  54% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G  131M  1.8G   7% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  5.7M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [3]:
os.environ['HF_HOME'] = '/run/cache/'

In [4]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-07 04:58:58--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-07 04:58:58 (42.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [5]:
import requests
import minsearch

In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
!nvidia-smi

Sun Jul  7 04:59:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   27C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
from huggingface_hub import login

In [11]:
login(token=os.environ['HF_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /run/cache/token
Login successful


In [20]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig

In [21]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
model.config.pad_token_id = tokenizer.eos_token_id

In [28]:
from transformers import pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
    QUESTION: {question}
    
    CONTEXT: {context}
    
    ANSWER: """
    context = ""
    for doc in search_results:
        context += f"{doc['question']}\n{doc['text']}\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = generator(prompt, max_length=500, temperature=0.7, top_p=0.95, num_return_sequences=1, truncation=True, do_sample=True)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()


In [30]:
rag("I just discovered the course. Can I still join it?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Yes, even if you don't register, you're still eligible to submit the homeworks.\n    \n    BE SURE TO READ THE POST TITLE CAREFULLY AND THE REST OF THE COMMENTS BEFORE MAKING A POST. MANY TIMES THE AN"

In [31]:
rag("when will the course start")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'> 2023-12-25 06:37:11\n    > \n    > Course - When will the course start?\n    > The purpose of this document is to capture frequently asked technical questions\n    > The exact day and hour of the course will be 15th Jan 2024 at 17h0'

In [32]:
rag("do i need github codespaces")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'QUESTION: Do I need to delete my instance in Google Cloud?\n    ANSWER: Nope. Do not delete your instance in Google Cloud platform. Otherwise, you have to do this twice for the week 1 readings.\n    \n    QUESTION: Do I need to use GitHub codespaces?\n    ANSWER: No, you can use a local environment.\n    \n    QUESTION: Is GitHub codespaces an alternative to using cli/git bash to ingest the data and create a docker file?\n    ANSWER: GitHub codespaces'